# Approximate Inference

We have recently demonstrated that exact inference is rather non-trivial. It's conceptually straightforward, but practically difficult and very expensive, scaling very badly with the number of variables. It also **fails completely** for continuous variables where the calculations involved are essentially intractable. For these reasons it very rare to use this approach in practice. So what do we do instead? The dominant class of algorithm for inference on Bayesian networks is based on *sampling* and we will now study two approaches to sampling. The first is the *direct sampling* method which is straightforward but inefficient. The second is *Markov Chain sampling* and specifically *Gibbs sampling*. This is the dominant method in the practical application of Bayesian networks.

## Direct Sampling

Imagine we have a coin and we want to know its distribution of heads vs tails if we toss it. How can we do this? We cannot determine the distribution easily from first principle, although we typically make some assumptions about the symmetry of the coin to conclude that $P(H)=P(T)=0.5$. How could we verify that our assumptions were correct though?

There is a very simple answer to this. We *sample* the distribution by tossing the coin a large number of times ($N$) and counting how many times each of the two outcomes occurs ($N_H, N_T$). We then estimate the distribution as $P(H)=N_H/N$ and $P(T)=N_T/N$. If the number of heads is roughly equal to the number of tails then we have approximately verified that that coin is fair and that $P(H)=P(T)$. We can simulate a fair coin using the Binomial Distribution with $p=0.5$

In [2]:
import numpy as np
ntosses = 1000
nheads = np.random.binomial(ntosses,0.5)
print(f"P(H)={nheads/ntosses}  P(T)={1-nheads/ntosses}")

P(H)=0.504  P(T)=0.496


Sampling can provide very accurate estimate of a distribution provided sufficiently many samples are taken. Here's another example of a Monte Carlo Pi estimator


In [11]:
npts = 100000000
x = np.random.uniform(-1., 1., size=(npts,2))
in_circle = x[np.where(x[:,0]*x[:,0]+x[:,1]*x[:,1] <= 1)]
est_pi = 4*in_circle.shape[0]/npts
print(est_pi)

3.14167924



How do we apply this idea to Bayesian networks, where we have a chain of conditional distributions? Let's illustrate via an example.

Consider our 


* Draw one sample from $P(G) = (0.4,0.6). Answer is (say) False.
* Draw one sample from $P(M) = (0.75,0.25). Answer is (say) True.
* Draw one sample from $P(F\vert m) = (0.8,0.2)$. Answer is True.
* Draw one sample from $P(P\vert m) = (0.55,0.45)$. Answer is False.

We have now drawn one sample from the joint distribution and obtained (False,True,True,False). Given a large number of such samples, we can compute anything we want. Let's try it out. We will code this up very explicitly which will be slightly inefficient but we want to maintain transparency whilst we are learning.

In [3]:
# It will help us to have the distributions. We will set these such that index 0 corresponds to False and 1 to True
P_G = np.array([0.6,0.4])
P_M = np.array([0.25,0.75])
P_F_M = np.array([[0.8,0.4],[0.2,0.6]])
P_P_M = np.array([[0.55,0.45],[0.45,0.55]])

In [35]:
N = 50000
Trace = np.zeros([N,4])
for i in range(N):
    # Draw one sample from P(G)
    G = np.random.binomial(1,P_G[1])
    # Draw one sample from P(M)
    M = np.random.binomial(1,P_M[1])
    # Draw one sample from P(F|M)
    F = np.random.binomial(1,P_F_M[1,M])
    # Draw one sample from P(P|M)
    P = np.random.binomial(1,P_P_M[1,M])
    Trace[i:] = np.array([G,M,F,P])

print(Trace)

[[0. 1. 1. 0.]
 [0. 1. 0. 0.]
 [1. 1. 0. 0.]
 ...
 [1. 0. 1. 0.]
 [1. 1. 1. 0.]
 [0. 1. 0. 0.]]


Let's do some basic stats on the Trace

In [36]:
print(f"P_G = {Trace[:,0].sum()/N} (expected 0.4)")
print(f"P_M = {Trace[:,1].sum()/N} (expected 0.75)")
print(f"P_F = {Trace[:,2].sum()/N} (expected 0.5)")
print(f"P_P = {Trace[:,3].sum()/N} (expected 0.525)")

P_G = 0.40244 (expected 0.4)
P_M = 0.74812 (expected 0.75)
P_F = 0.49832 (expected 0.5)
P_P = 0.52584 (expected 0.525)


Pretty good! Can we now estimate some other quantities? For example, how about estimating $P(M\vert F)$?

First of all, we can do this by hand via Bayes' theorem: $P(M\vert F) = (F\vert M)P(F)$. We know all these quantities so we can calculate $P(m\vert f) = P(f\vert m)P(m)/p(f) = 0.6\times 0.75/0.5 = 0.9$.

How do we get this from the trace?

To find $P(p\vert f)$ we need to
* Find the rows in the Trace where the condition $f$ is True.
* Then compute the proportion of those rows for which $p$ is True.

In [79]:
# Find the rows where F is True
FTrue = Trace[np.where(Trace[:,2]==1)]
# Add up column 1 and divide by the number of samples where F is True
Pm = FTrue[:,1].sum()/FTrue.shape[0]
print(f"P(m|f) = {Pm} (0.9 expected)")


P(m|f) = 0.9007013069811922 (0.9 expected)


This is an example of **rejection sampling**. We have thrown away all of the rows in the Trace for which $F$ was false. We may consider this to be a waste; however, if we want to ask general question of the distribution, the full trace in which all variables are drawn allows us to do so.

Rejection sampling is very general and it can be shown that the standard deviation in the error in each probability is proportional to $1/\sqrt{N}$. However, if we have specific questions we want to answer and are time or compute-limited, then we may wish for something more efficient. This will be our next topic.


# Importance Sampling

In importance sampling, we generate only those values which we need. This only works on conditioning variables.

In [4]:
N = 10000
Trace = np.zeros([N,4])
for i in range(N):
    # Draw one sample from P(G)
    G = np.random.binomial(1,P_G[1])
    # Draw one sample from P(M)
    #M = np.random.binomial(1,P_M[1])
    M = 1
    # Draw one sample from P(F|M)
    F = np.random.binomial(1,P_F_M[1,M])
    # Draw one sample from P(P|M)
    P = np.random.binomial(1,P_P_M[1,M])
    Trace[i:] = np.array([G,M,F,P])


In [5]:
print(f"P_G = {Trace[:,0].sum()/N}")
print(f"P_M = {Trace[:,1].sum()/N}")
print(f"P_F_M = {Trace[:,2].sum()/N}")
print(f"P_P_M = {Trace[:,3].sum()/N}")



P_G = 0.4024
P_M = 1.0
P_F_M = 0.6002
P_P_M = 0.548


# Likelihood Reweighting

First look at conditioning on M

In [6]:
N = 10000
Trace = np.zeros([N,5])
for i in range(N):
    w = 1.0
    # Draw one sample from P(G)
    G = np.random.binomial(1,P_G[1])
    # Draw one sample from P(M)
    #M = np.random.binomial(1,P_M[1])
    M = 1
    w *= P_M[M]
    # Draw one sample from P(F|M)
    F = np.random.binomial(1,P_F_M[1,M])
    # Draw one sample from P(P|M)
    P = np.random.binomial(1,P_P_M[1,M])
    Trace[i:] = np.array([G,M,F,P,w])

In [7]:
print(Trace)

[[0.   1.   1.   1.   0.75]
 [1.   1.   0.   0.   0.75]
 [1.   1.   1.   0.   0.75]
 ...
 [0.   1.   1.   0.   0.75]
 [0.   1.   1.   1.   0.75]
 [1.   1.   1.   0.   0.75]]


In [10]:
print(f"P_G = {np.multiply(Trace[:,0],Trace[:,4]).sum()/Trace[:,4].sum()}")
print(f"P_M = {np.multiply(Trace[:,1],Trace[:,4]).sum()/Trace[:,4].sum()}")
print(f"P_F_M = {np.multiply(Trace[:,2],Trace[:,4]).sum()/Trace[:,4].sum()}")
print(f"P_P_M = {np.multiply(Trace[:,3],Trace[:,4]).sum()/Trace[:,4].sum()}")



P_G = 0.4013
P_M = 1.0
P_F_M = 0.601
P_P_M = 0.5571


A second example - this time conditioning on those people who are programmers. 

In [11]:
N = 10000
Trace = np.zeros([N,5])
for i in range(N):
    w = 1.0
    # Draw one sample from P(G)
    #G = np.random.binomial(1,P_G[1])
    G = 0
    w *= P_G[G]
    # Draw one sample from P(M)
    M = np.random.binomial(1,P_M[1])
    # Draw one sample from P(F|M)
    F = np.random.binomial(1,P_F_M[1,M])
    # Draw one sample from P(P|M)
    #P = np.random.binomial(1,P_P_M[1,M])
    P = 1
    w *= P_P_M[P,M]
    Trace[i:] = np.array([G,M,F,P,w])

In [12]:
print(f"P_G = {np.multiply(Trace[:,0],Trace[:,4]).sum()/Trace[:,4].sum()}")
print(f"P_M = {np.multiply(Trace[:,1],Trace[:,4]).sum()/Trace[:,4].sum()}")
print(f"P_F_M = {np.multiply(Trace[:,2],Trace[:,4]).sum()/Trace[:,4].sum()}")
print(f"P_P_M = {np.multiply(Trace[:,3],Trace[:,4]).sum()/Trace[:,4].sum()}")



P_G = 0.0
P_M = 0.7842771130249218
P_F_M = 0.517024235957625
P_P_M = 1.0
